# Major Depressive Disorder Diagnosis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# HRV 데이터셋 불러오기
hrv_df = pd.read_csv('E:/RESEARCH/Datasets/HRV/HRV_samsung/HRV_REV_all.csv', sep=',')
hrv_df.head()

,sub,VISIT,disorder,age,gender,HAMD,HAMA,PDSS,ASI,APPQ,...,cRMSSD,cVLF,cLF,cHF,cLF/HF,cPOWER,cHR,cRESP,cSC,cTEMP
0,E001,4,2,23,1,2,2,1,12,22,...,41.544667,190.107000,298.508333,206.862333,1.284000,695.477333,65.707000,14.054333,3.911333,34.998000
1,E001,5,2,23,1,12,7,0,12,24,...,39.825333,143.756667,115.695333,202.602667,0.585000,462.054667,69.040000,14.117333,6.255000,35.544333
2,E002,1,2,38,1,14,17,14,31,122,...,20.052000,22.006000,50.182000,32.529333,2.499333,104.717000,92.789333,11.013333,0.945667,35.086000
3,E002,2,2,38,1,13,36,16,32,139,...,20.201667,55.579000,84.441000,18.754000,5.803000,158.774333,85.980667,12.608333,0.785667,36.141000
4,E002,3,2,38,1,7,10,11,23,70,...,74.788000,182.229000,530.565667,546.574000,1.685667,1259.368667,84.368667,14.285667,0.648000,35.879000


In [7]:
hrv_df.shape

(479, 93)

HRV measuring steps
* b1 - s - b2 - r - b3 - c
* Each has following variables (Total 13 variables)
> SDNN, NN50, PNN50, RMSSD, VLF, LF, HF, LF/HF, POWER, HR, RESP, SC, TEMP

In [65]:
## Separating HRV dataset by experimental steps.
hrv = hrv_df
hrv_b1 = hrv_df.filter(regex='^b1')
hrv_s = hrv_df.filter(regex='^s')
hrv_b2 = hrv_df.filter(regex='^b2')
hrv_r = hrv_df.filter(regex='^r')
hrv_b3 = hrv_df.filter(regex='^b3')
hrv_c = hrv_df.filter(regex='^c')

In [66]:
print("HRV baseline #1 shape is:", hrv_b1.shape[1])
print("HRV stress shape is:", hrv_s.shape[1])
print("HRV baseline #2 shape is:", hrv_b2.shape[1])
print("HRV rest shape is:", hrv_r.shape[1])
print("HRV baseline #3 shape is:", hrv_b3.shape[1])
print("HRV c shape is:", hrv_b1.shape[1])

HRV baseline #1 shape is: 13
HRV stress shape is: 14
HRV baseline #2 shape is: 13
HRV rest shape is: 13
HRV baseline #3 shape is: 13
HRV c shape is: 13


In [70]:
hrv_s = hrv_s.drop(columns=['sub'])

In [71]:
## Renaming the columns for further calculation.
## We need to generate new dataframes to compare the phases.
hrv_variables = ["SDNN", "NN50", "PNN50", "RMSSD", "VLF", "LF", "HF", "LF/HF", "POWER", "HR", "RESP", "SC", "TEMP"]
hrv_b1_rename = hrv_b1.set_axis(["SDNN", "NN50", "PNN50", "RMSSD", "VLF", "LF", "HF", "LF/HF", "POWER", "HR", "RESP", "SC", "TEMP"], axis=1)
hrv_b2_rename = hrv_b2.set_axis(["SDNN", "NN50", "PNN50", "RMSSD", "VLF", "LF", "HF", "LF/HF", "POWER", "HR", "RESP", "SC", "TEMP"], axis=1)
hrv_b3_rename = hrv_b3.set_axis(["SDNN", "NN50", "PNN50", "RMSSD", "VLF", "LF", "HF", "LF/HF", "POWER", "HR", "RESP", "SC", "TEMP"], axis=1)
hrv_s_rename = hrv_s.set_axis(["SDNN", "NN50", "PNN50", "RMSSD", "VLF", "LF", "HF", "LF/HF", "POWER", "HR", "RESP", "SC", "TEMP"], axis=1)
hrv_r_rename = hrv_r.set_axis(["SDNN", "NN50", "PNN50", "RMSSD", "VLF", "LF", "HF", "LF/HF", "POWER", "HR", "RESP", "SC", "TEMP"], axis=1)
hrv_c_rename = hrv_c.set_axis(["SDNN", "NN50", "PNN50", "RMSSD", "VLF", "LF", "HF", "LF/HF", "POWER", "HR", "RESP", "SC", "TEMP"], axis=1)

In [49]:
hrv_b2.columns

Index(['b2SDNN', 'b2NN50', 'b2PNN50', 'b2RMSSD', 'b2VLF', 'b2LF', 'b2HF',
       'b2LF/HF', 'b2POWER', 'b2HR', 'b2RESP', 'b2SC', 'b2TEMP'],
      dtype='object')

In [50]:
hrv_s.columns

Index(['sub', 'sSDNN', 'sNN50', 'sPNN50', 'sRMSSD', 'sVLF', 'sLF', 'sHF',
       'sLF/HF', 'sPOWER', 'sHR', 'sRESP', 'sSC', 'sTEMP'],
      dtype='object')

In [51]:
hrv_r.columns

Index(['rSDNN', 'rNN50', 'rPNN50', 'rRMSSD', 'rVLF', 'rLF', 'rHF', 'rLF/HF',
       'rPOWER', 'rHR', 'rRESP', 'rSC', 'rTEMP'],
      dtype='object')

In [52]:
hrv_b1

,b1SDNN,b1NN50,b1PNN50,b1RMSSD,b1VLF,b1LF,b1HF,b1LF/HF,b1POWER,b1HR,b1RESP,b1SC,b1TEMP
0,53.113667,14.333333,0.233000,56.151000,100.156000,99.322333,315.010667,0.319333,514.488667,60.964333,15.761000,0.841333,34.684333
1,45.919000,13.666667,0.210000,46.576000,101.827333,204.858333,235.766333,0.914667,542.452000,64.646333,15.483667,0.589000,35.623667
2,26.377667,0.000000,0.000000,15.102000,131.038333,88.803333,42.704000,2.854667,262.546000,92.641333,17.677667,0.806333,35.167333
3,24.147333,0.000000,0.000000,13.519333,132.222000,177.344000,47.517000,4.686333,357.083333,90.762000,18.256000,0.618333,36.169000
4,79.460333,6.000000,0.066333,87.518667,84.822667,326.738667,303.335333,0.817333,714.897000,91.826667,17.347000,0.518000,35.962000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,20.610300,0.000000,0.000000,15.115700,24.712300,41.573700,25.551700,1.675000,91.837000,77.549000,17.473000,0.368667,33.354300
475,652.454000,31.000000,0.470000,971.994000,139081.000000,113304.000000,56593.500000,1.344330,308979.000000,95.077000,13.471700,2.952000,33.215300
476,254.235000,19.333300,0.209333,369.508000,30359.800000,125554.000000,27678.300000,2.081330,183592.000000,101.815000,13.811000,0.234333,33.601000
477,27.793700,1.333330,0.016667,27.832700,73.146000,84.824000,112.416000,1.155670,270.386000,80.856300,14.411700,0.215333,32.315300


,b2SDNN,b2NN50,b2PNN50,b2RMSSD,b2VLF,b2LF,b2HF,b2LF/HF,b2POWER,b2HR,b2RESP,b2SC,b2TEMP
0,64.186000,15.000000,0.243333,63.772667,161.622000,445.318000,537.075333,0.926333,1144.015000,61.131333,14.242000,5.967000,34.539000
1,58.021667,14.333333,0.219667,50.353333,240.611667,293.423333,396.367000,0.720667,930.402000,64.512667,13.017667,7.806000,34.976000
2,21.981333,0.000000,0.000000,13.459000,41.562667,46.186333,23.762000,2.118333,111.511333,91.731333,16.408000,0.994000,35.184667
3,23.562667,0.000000,0.000000,14.762667,82.235000,73.548000,34.330000,1.767333,190.112667,86.505667,17.765333,0.648667,36.270333
4,25.840000,1.333333,0.015333,28.100667,34.566333,97.600000,43.935667,1.990333,176.102333,86.466667,17.294667,0.548333,36.020667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,20.542000,0.333333,0.004333,15.470700,12.461000,59.848000,22.655300,2.061330,94.965000,77.033000,17.603300,0.409667,33.723000
475,469.753000,38.000000,0.438333,686.048000,131428.000000,310104.000000,55214.000000,4.494670,496745.000000,111.128000,12.714000,5.743330,31.337700
476,663.841000,35.666700,0.436333,975.182000,176049.000000,124518.000000,38474.900000,2.318670,339042.000000,117.155000,14.009300,1.363330,33.460000
477,26.469000,1.333330,0.017333,25.314000,66.709000,182.448000,40.881000,4.669670,290.038000,77.254000,14.499300,0.405000,33.220000
